# Neareset Neighbor Search
## October 28th, 2021
### Overview: Use Nearest Neighbors to classify data

In [1]:
import numpy as np
from scipy.spatial import KDTree
from scipy import stats

### Given an mxk training set X, use an exhaustive search to find the nearest point in the set to a target point z

In [2]:
def exhaustive_search(X, z):
    """Solve the nearest neighbor search problem with an exhaustive search.

    Parameters:
        X ((m,k) ndarray): a training set of m k-dimensional points.
        z ((k, ) ndarray): a k-dimensional target point.

    Returns:
        ((k,) ndarray) the element (row) of X that is nearest to z.
        (float) The Euclidean distance from the nearest neighbor to z.
    """
    #dists is the array of distances from a point in x to the target point
    dists = np.linalg.norm((X-z),axis=1)
    
    #index of row of X with minimum distance, and the minimum distance
    index = np.argmin(dists)
    minDist = np.min(dists)
    
    #returning that row and the distance
    return X[index], minDist

### Creation of the KDTNode and KDT (k-dimensional tree) classes

In [3]:
class KDTNode:
    """Node class for K-D Trees.

    Attributes:
        left (KDTNode): a reference to this node's left child.
        right (KDTNode): a reference to this node's right child.
        value ((k,) ndarray): a coordinate in k-dimensional space.
        pivot (int): the dimension of the value to make comparisons on.
    """
    def __init__(self,x):
        #checking x is an np.ndarray
        if type(x) != np.ndarray:
            raise TypeError('x must be a np array')
        #otherwise saving x as value and initializng right, left, and pivot as None
        else:
            self.value = x
            self.right = None
            self.left = None
            self.pivot = None

In [4]:
class KDT:
    """A k-dimensional binary tree for solving the nearest neighbor problem.

    Attributes:
        root (KDTNode): the root node of the tree. Like all other nodes in
            the tree, the root has a NumPy array of shape (k,) as its value.
        k (int): the dimension of the data in the tree.
    """
    def __init__(self):
        """Initialize the root and k attributes."""
        self.root = None
        self.k = None

    def find(self, data):
        """Return the node containing the data. If there is no such node in
        the tree, or if the tree is empty, raise a ValueError.
        """
        def _step(current):
            """Recursively step through the tree until finding the node
            containing the data. If there is no such node, raise a ValueError.
            """
            if current is None:                     # Base case 1: dead end.
                raise ValueError(str(data) + " is not in the tree")
            elif np.allclose(data, current.value):
                return current                      # Base case 2: data found!
            elif data[current.pivot] < current.value[current.pivot]:
                return _step(current.left)          # Recursively search left.
            else:
                return _step(current.right)         # Recursively search right.

        # Start the recursive search at the root of the tree.
        return _step(self.root)

    
    def insert(self, data):
        """Insert a new node containing the specified data.

        Parameters:
            data ((k,) ndarray): a k-dimensional point to insert into the tree.

        Raises:
            ValueError: if data does not have the same dimensions as other
                values in the tree.
            ValueError: if data is already in the tree
        """
        def _step2(current):
            """Recursively steps through the tree until finding the node which should become the parent of the new node"""
            #checking for duplicate data
            if np.allclose(data,current.value):
                raise ValueError("Cannot have duplicate data")
            elif current.right != None:
                if np.allclose(data,current.right.value):
                    raise ValueError("Cannot have duplicate data")
            elif current.left != None:
                if np.allclose(data,current.left.value):
                    raise ValueError("Cannot have duplicate data")
            
            #stepping through to next appropriate node
            if data[current.pivot] < current.value[current.pivot] and current.left != None:
                return _step2(current.left)
            elif data[current.pivot] < current.value[current.pivot] and current.left == None:
                return current, 'left'
            elif data[current.pivot] > current.value[current.pivot] and current.right != None:
                return _step2(current.right)
            else:
                return current, 'right'
            
        #if no root exists, insert new node as root with pivot 0, and set k to the dimension of data
        if self.root == None:
            insertNode = KDTNode(data)
            insertNode.pivot = 0
            self.k = len(insertNode.value)
            self.root = insertNode
        
        #if a root already exists
        else:
            #raise ValueError if data is not in R^k
            if data.shape != (self.k,):
                raise ValueError("Data must be same dimension as the root")
                
            #else insert   
            else:
                parent, direction = _step2(self.root)
                insertNode = KDTNode(data)
                if direction == 'left':
                    parent.left = insertNode
                else:
                    parent.right = insertNode
                insertNode.pivot = (parent.pivot + 1) % self.k
                

    def query(self, z):
        """Find the value in the tree that is nearest to z.

        Parameters:
            z ((k,) ndarray): a k-dimensional target point.

        Returns:
            ((k,) ndarray) the value in the tree that is nearest to z.
            (float) The Euclidean distance from the nearest neighbor to z.
        """
        def search(current,nearest,dstar):
            """This intermediary function moves through the tree to search and gets called recursively"""
            if current == None:
                return nearest, dstar
            else:
                x = current.value
                i = current.pivot
                d = np.linalg.norm(x-z)
                if d < dstar:
                    nearest = current
                    dstar = d
                if z[i] < x[i]:
                    nearest, dstar = search(current.left,nearest,dstar)
                    if z[i] + dstar >= x[i]:
                        nearest, dstar = search(current.right,nearest,dstar)
                else:
                    nearest, dstar = search(current.right,nearest,dstar)
                    if z[i] - dstar <= x[i]:
                        nearest, dstar = search(current.left,nearest,dstar)
            return nearest,dstar
        
        node,dstar = search(self.root,self.root,la.norm(self.root.value - z))
        return node.value, dstar
        

    def __str__(self):
        """String representation: a hierarchical list of nodes and their axes.

        Example:                           'KDT(k=2)
                    [5,5]                   [5 5]   pivot = 0
                    /   \                   [3 2]   pivot = 1
                [3,2]   [8,4]               [8 4]   pivot = 1
                    \       \               [2 6]   pivot = 0
                    [2,6]   [7,5]           [7 5]   pivot = 0'
        """
        if self.root is None:
            return "Empty KDT"
        nodes, strs = [self.root], []
        while nodes:
            current = nodes.pop(0)
            strs.append("{}\tpivot = {}".format(current.value, current.pivot))
            for child in [current.left, current.right]:
                if child:
                    nodes.append(child)
        return "KDT(k={})\n".format(self.k) + "\n".join(strs)

### Creation of the classifier object

In [5]:
class KNeighborsClassifier:
    """A k-nearest neighbors classifier that uses SciPy's KDTree to solve
    the nearest neighbor problem efficiently.
    """
    from scipy.spatial import KDTree
    from scipy import stats
    
    def __init__(self,n_neighbors):
        """Accepts an integer n_neighbors as the number of neighboring nodes to determine the label of a target"""
        self.k = n_neighbors
        
    def fit(self, X, y):
        """
        Loads a scipy KDTree with data of an array of points X and saves an array of labels y as an attribute
        X (m x k NumPy Array) - the training set
        y (m x 1 NumPy Array) - the training labels
        """
        self.tree = KDTree(X)
        self.y = y
        
    def predict(self, z):
        """
        Accepts 1-dimensional array z with k entries and returns the most common label of points around z
        """
        indices = self.tree.query(z, k = self.k)[1]
        label = stats.mode(self.y[indices])[0][0]
        return label

### An example implementation of the classifier on a subset of an mnist data set

In [6]:
def implementation(n_neighbors, filename="mnist_subset.npz"):
    """Extract the data from the given file. Load a KNeighborsClassifier with
    the training data and the corresponding labels. Use the classifier to
    predict labels for the test data. Return the classification accuracy, the
    percentage of predictions that match the test labels.

    Parameters:
        n_neighbors (int): the number of neighbors to use for classification.
        filename (str): the name of the data file. Should be an npz file with
            keys 'X_train', 'y_train', 'X_test', and 'y_test'.

    Returns:
        (float): the classification accuracy.
    """
    #Extracting data
    data = np.load(filename)
    X_train = data["X_train"].astype(float)
    y_train = data["y_train"]
    X_test = data["X_test"].astype(float)
    y_test = data["y_test"]
    
    #making a classifier and predicting the test data
    classifier = KNeighborsClassifier(n_neighbors)
    classifier.fit(X_train,y_train)
    
    #iterating through the test data and comparing to the correct y test results
    accuracy = 0
    
    for i, image in enumerate(X_test):
        if classifier.predict(image) == y_test[i]:
            accuracy += 1
    
    #calculating and returning the classification accuracy
    accuracy = accuracy/len(y_test)
    return accuracy

#### Implementation tested on different numbers of nearest neighbors:

In [7]:
for n in [1,2,3,4,5,10]:
    print(implementation(n))

0.934
0.902
0.91
0.91
0.906
0.906
